# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [ ]:
import xlwings as xw


In [1]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import csv

In [ ]:
book_1 = xw.Book("себестоимостьА_в1.xlsx")
sheet = book_1.sheets["Рецептура"]
cons = sheet.range("G7:O10").options(np.array).value
U_p = sheet.range("G14:O14").options(np.array).value
C_p = cons * U_p
C_p[np.isnan(C_p)] = 0
C_p = np.round(np.sum(C_p, axis=1).tolist(), 2)
C_p

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [ ]:
sheet.range("T7:T10").options(transpose=True).value = C_p
sheet.range("T6").value = "Себестоимость"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
sheet.range("T4:T6").column_width = 15 
sheet.range("T4:T6").color = 255,192,0 
sheet.range("T4:T6").api.Font.ColorIndex = 2   
sheet.range("T4:T6").api.Font.Size = 11
sheet.range("T4:T10").api.Borders.Weight = 2
sheet.range("T4:T6").api.HorizontalAlignment = -4108
sheet.range("T4:T6").api.VerticalAlignment = -4108
sheet.range("T7:T10").color = 255,255,167 
sheet.range("T4:T6").merge()

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
sheet.range("U7:U10").formula = "=SUMPRODUCT(G7:O7, $G$14:$O$14)"

## Лабораторная работа 7.1

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
reviews = pd.read_csv('reviews_sample.csv', delimiter=',')
recipes = pd.read_csv('recipes_sample.csv', delimiter= ',')
recipes = recipes[['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients']]
print(reviews.head())
print(recipes.head())

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [6]:
from pandas import ExcelWriter
writer = ExcelWriter(r'recipes.xlsx')

recipe = recipes.sample(frac = 0.05)
review = reviews.sample(frac = 0.05)
recipe.to_excel(writer, encoding='utf8',sheet_name="Рецепты")
review.to_excel(writer, encoding='utf8',sheet_name="Отзывы")
writer.save()

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
sheet = xw.Book('recipes.xlsx').sheets['Рецепты']
sheet.range('H1').value = "second_assign"

sheet.range("H2:H1501").options(transpose=True).value = (recipes["minutes"] * 60).to_numpy()
sheet.autofit()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
sheet.range('I1').value = "seconds_formula"
sheet.range("I2:I1501").formula = "=D2*60"
sheet.autofit()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
sheet.range('H1').api.Font.Bold = True
sheet.range('I1').api.Font.Bold = True
sheet.range('H1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
sheet.range('I1').api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
def coloring(a):
    c = (a[2], a[1], a[0])
    st = '%02x%02x%02x' % c
    value = int(st, 16)
    return value

rgb = sheet.api.Range("$D2:$D1501")
rgb.FormatConditions.Add(1, 6, '=5')
rgb.FormatConditions(1).Interior.Color=coloring((0,255,0))
rgb.FormatConditions.Add(1, 1, '=5',"=10")
rgb.FormatConditions(2).Interior.Color=coloring((255,255,0))
rgb.FormatConditions.Add(1, 5,"=10")
rgb.FormatConditions(3).Interior.Color=coloring((255,0,0))

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
sheet.range('R1').value = 'n_reviews'
sheet.range('R2').formula = '=COUNTIF(C2:C1501, "<>")'
sheet.range('R2').expand('down').formula = sheet.range('R2').formula

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate():
    reviews_sheet = xw.Book('recipes.xlsx').sheets['Отзывы']
    recipes_sheet = xw.Book('recipes.xlsx').sheets['Рецепты']
    reviews = reviews_sheet.range('B2:B1501').value
    recipes = reviews_sheet.range('C2:C1501').value
    for i in range(len(reviews)):
        if not (0 <= reviews[i] <= 5):
            reviews_sheet.range(f'B{i+2}:C{i+2}').color = (255, 0, 0)
        if recipes[i] not in recipes_sheet.range('A2:A1501').value:
            reviews_sheet.range(f'B{i+2}:C{i+2}').color = (255, 0, 0)

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
with open('recipes_model.csv', 'r', encoding='utf-8') as file:
    reader = csv.reader(file, delimiter="\t")
    lst = list(reader)

new_book = xw.Book()
sheet = new_book.sheets.add('Модель')
sheet.range('A2:I18').value = lst
new_book.save('recipes_model.xlsx')

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:

sheet = xw.Book('recipes_model.xlsx').sheets['Модель']
last_row = sheet.range('A' + str(sheet.cells.last_cell.row)).end('up').row
formula = '=B2&" "&UPPER(C2)'
sheet.range('J2').value = formula
sheet.range('J2').api.autofill(sheet.range(f'J2:J{last_row}').api, 0)
sheet.range('J2:J' + str(last_row)).api.autofit()


11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
headers = sheet.range('A1:J1')
headers.color = (0, 204, 255)
headers.api.Font.Bold = True
sheet.autofit('c')
sheet.range('A1:J1').api.AutoFilter()

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [ ]:
open_book = xw.Book("recipes_model.xlsx")

lst = pd.read_csv("recipes_model.csv")
stat = lst.groupby("Сущность")["Атрибут"].agg("count")
open_book.sheets.add("Статистика").range("A1").value = stat
stat.plot(kind="bar")
open_book.sheets["Статистика"].pictures.add(stat.plot().figure, name="Статистика", update=True, left=open_book.sheets["Статистика"].range("E2").left, top=open_book.sheets["Статистика"].range("E2").top)
